# Feature Engineering test ground
![Reverse Engineer](https://crystal-cdn3.crystalcommerce.com/photos/6337507/en_UJy7SBBy4F.png)

In [2]:
import numpy as np
import pandas as pd
from master_transmuter import *

In [4]:
cd ..

C:\Users\alexg\Galvanize\mystic-speculation


In [7]:
all_cards_df = pd.read_csv('scrape/all_vintage_cards.csv', low_memory=False)

In [10]:
all_cards_df['toughness'].unique()

array([nan, '8', '2', '5', '*', '3', '1+*', '1', '6', '10', '12', '15',
       '4', '7', '16', '0', '11', '9', '13', '7-*', '2+*', '14', '-1'],
      dtype=object)

In [28]:
tough_mask = all_cards_df['toughness'].apply(lambda x: '*' in str(x))
power_mask = all_cards_df['power'].apply(lambda x: '*' in str(x))
all_cards_df[['name', 'power', 'toughness']][tough_mask | power_mask]

,name,power,toughness
17,Lord of Extinction,*,*
21,Tarmogoyf,*,1+*
57,Lord of Extinction,*,*
61,Tarmogoyf,*,1+*
263,Consuming Aberration,*,*
439,Crackling Drake,*,4
719,Psychosis Crawler,*,*
734,Darksteel Juggernaut,*,*
741,Zendikar Incarnate,*,4
747,Rubblehulk,*,*
